In [1]:
import psycopg2

import copy
import random
import numpy as np
import pandas as pd

import joblib
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.utils.data import Subset, DataLoader

from Dataset.Embedding_Dataset import Embedding_Dataset
from Model.Embedding import Embedding

from Dataset.Apartment_Complex_Dataset import Apartment_Complex_Dataset
from Model.LSTM import LSTM
from Model.GRU import GRU
from Model.Transformer import Transformer

from Dataset.Dong_Dataset import Dong_Dataset
from Model.LSTM_Attention import LSTMAttention
from Model.GRU_Attention import GRUAttention
from Model.Transformer_Attention import TransformerAttention

from utils import RMSE, save_train_val_losses

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

table_1 = pd.read_csv('../데이터/Table/table_1.csv') 
table_2 = pd.read_csv('../데이터/Table/table_2.csv') 
table_3 = pd.read_csv('../데이터/Table/table_3.csv') 

In [ ]:
epochs = 10000

embedding_dim = 1024

transformer_lr = 1e-4
transformer_batch = 128
transformer_window_size = 30

transformer_att_lr = 1e-4
transformer_att_batch = 1
transformer_att_hidden_dim = 1024
transformer_att_window_size = 30

In [ ]:
model = torch.load('../데이터/Checkpoint/embedding_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_6_e1_128_e2_128_e3_512_emb_1024_d1_512_d2_256_d3_128.pth', map_location=torch.device('cpu'))
dataset = Apartment_Complex_Dataset(model, table_1, table_2, table_3, embedding_dim, transformer_window_size, 'DL', 'TRAIN', DEVICE)

train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1
dataset_length = len(dataset)
train_size = int(train_ratio * dataset_length)
train_indices = range(0, train_size)
val_size = int(val_ratio * dataset_length)
val_indices = range(train_size, train_size + val_size)
test_size = int(test_ratio * dataset_length)
test_indices = range(train_size + val_size, dataset_length)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

In [ ]:
model = Transformer(embedding_dim, transformer_window_size, 1, 2, 2).to(DEVICE)
criterion = RMSE()
optimizer = torch.optim.Adam(model.parameters(), lr=transformer_lr, weight_decay=transformer_weight_decay)

train_losses = []
val_losses = []
best_val_loss = float('inf')
consecutive_val_loss_increases = 0
max_consecutive_val_loss_increases = 3

for epoch in range(transformer_epochs):
    model.train()
    total_train_loss = 0
    for i, data in enumerate(train_dataloader):
        src = data[0].to(DEVICE)
        trg = data[1].to(DEVICE)

        if (trg[0] != 0):
            src_mask = model.generate_square_subsequent_mask(src.shape[1]).to(src.device)
            output = model(src, src_mask)

            train_loss = criterion(output[0], trg)
            total_train_loss += train_loss.item()

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for data in val_dataloader:
            src = data[0].to(DEVICE)
            trg = data[1].to(DEVICE)

            if (trg[0] != 0):
                src_mask = model.generate_square_subsequent_mask(src.shape[1]).to(src.device)
                output = model(src, src_mask)

                val_loss = criterion(output[0], trg)
                total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)

    early_stop, consecutive_val_loss_increases = early_stop_1(val_losses, consecutive_val_loss_increases, max_consecutive_val_loss_increases)
    # early_stop, best_val_loss, consecutive_val_loss_increases = early_stop_2(avg_val_loss, best_val_loss, consecutive_val_loss_increases, max_consecutive_val_loss_increases)
    if early_stop:
        print(f'Epoch [{epoch+1}/{transformer_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f} \nEarly Stop Triggered!')
        torch.save(model, f'../데이터/Checkpoint/transformer_tr_{train_ratio}_lr_{transformer_lr}_wd_{transformer_weight_decay}_batch_{transformer_batch}_epochs_{epoch+1}_ws_{transformer_window_size}.pth')
        break

    print(f'Epoch [{epoch+1}/{transformer_epochs}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}')

torch.save(model, f'../데이터/Checkpoint/transformer_tr_{train_ratio}_lr_{transformer_lr}_wd_{transformer_weight_decay}_batch_{transformer_batch}_epochs_{epoch+1}_ws_{transformer_window_size}.pth')
plot_train_val_losses(train_losses, val_losses)

In [19]:
model = torch.load('../데이터/Checkpoint/embedding_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_6_e1_128_e2_128_e3_512_emb_1024_d1_512_d2_256_d3_128.pth', map_location=torch.device('cpu'))
dataset = Dong_Dataset(model, table_1, table_2, table_3, embedding_dim, transformer_att_window_size, 'TRAIN', DEVICE)

train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1
dataset_length = len(dataset)
train_size = int(train_ratio * dataset_length)
train_indices = range(0, train_size)
val_size = int(val_ratio * dataset_length)
val_indices = range(train_size, train_size + val_size)
test_size = int(test_ratio * dataset_length)
test_indices = range(train_size + val_size, dataset_length)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

In [20]:
transformer_model = torch.load('../데이터/Checkpoint/transformer_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_5_ws_3.pth', map_location=torch.device('cpu'))
model = TransformerAttention(transformer_model, transformer_att_hidden_dim, 1, DEVICE).to(DEVICE)
criterion = RMSE()
optimizer = torch.optim.Adam(model.parameters(), lr=transformer_att_lr, weight_decay=transformer_att_weight_decay)

train_losses = []
val_losses = []
best_val_loss = float('inf')
consecutive_val_loss_increases = 0
max_consecutive_val_loss_increases = 3

for epoch in range(transformer_att_epochs):
    model.train()
    total_train_loss = 0
    for data in train_dataloader:
        src = data[0][0].to(DEVICE)
        max_len = data[1][0].to(DEVICE)
        anw = data[2][0].to(DEVICE)
        trg = data[3][0].to(DEVICE)
        
        if len(anw)==0:
            continue
        
        # dong_loss = 0

        for index in anw:
            output = model(src, index, max_len)
            
            train_loss = criterion(output, trg[index])
            total_train_loss += train_loss.item()
            # dong_loss += loss.item()
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step() 

        # optimizer.zero_grad()
        # # dong_loss /= len(anw)
        # dong_loss = torch.tensor(dong_loss, requires_grad=True).to(DEVICE)
        # dong_loss.backward()
        # optimizer.step()
            
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for data in val_dataloader:
            src = data[0][0].to(DEVICE)
            max_len = data[1][0].to(DEVICE)
            anw = data[2][0].to(DEVICE)
            trg = data[3][0].to(DEVICE)

            if len(anw)==0:
                continue

            for index in anw:
                output = model(src, index, max_len)

                val_loss = criterion(output, trg[index])
                total_val_loss += val_loss.item()
                
    avg_val_loss = total_val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
            
    early_stop, consecutive_val_loss_increases = early_stop_1(val_losses, consecutive_val_loss_increases, max_consecutive_val_loss_increases)
    # early_stop, best_val_loss, consecutive_val_loss_increases = early_stop_2(avg_val_loss, best_val_loss, consecutive_val_loss_increases, max_consecutive_val_loss_increases)
    if early_stop:
        print(f'Epoch [{epoch+1}/{transformer_att_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f} \nEarly Stop Triggered!')
        torch.save(model, f'../데이터/Checkpoint/transformer_att_tr_{train_ratio}_lr_{transformer_att_lr}_wd_{transformer_att_weight_decay}_batch_{transformer_att_batch}_epochs_{epoch+1}_hdim_{transformer_att_hidden_dim}_ws_{transformer_att_window_size}.pth')
        break

    print(f'Epoch [{epoch+1}/{transformer_att_epochs}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}')

torch.save(model, f'../데이터/Checkpoint/transformer_att_tr_{train_ratio}_lr_{transformer_att_lr}_wd_{transformer_att_weight_decay}_batch_{transformer_att_batch}_epochs_{epoch+1}_hdim_{transformer_att_hidden_dim}_ws_{transformer_att_window_size}.pth')
plot_train_val_losses(train_losses, val_losses)

Epoch [1/1000], Train Loss: 3.826740, Val Loss: 12.337122
